#  Zeitreihenanalyse mit der Box-Jenkins-Methode

## Einleitung

Ziel dieser Analyse ist es, ein geeignetes ARIMA-Modell zu finden, das den zugrunde liegenden datengenerierenden Prozess einer Finanzzeitreihe (z. B. Aktienkurs) möglichst realitätsnah abbildet. Die Analyse erfolgt nach der **Box-Jenkins-Methode**, einem etablierten Verfahren zur Modellierung, Auswahl und Prognose von Zeitreihen.

### Was ist eine Zeitreihe?

Eine **Zeitreihe** ist eine Folge von Beobachtungen, die in regelmäßigen Abständen über die Zeit hinweg aufgezeichnet wird – beispielsweise tägliche Aktienkurse oder monatliche Arbeitslosenzahlen. Der Zweck der Zeitreihenanalyse ist es, **Strukturen wie Trends, Saisonalität und Autokorrelationen zu identifizieren**, um präzise Prognosen für zukünftige Werte zu erstellen.

### Warum Box-Jenkins?

Die **Box-Jenkins-Methode** konzentriert sich auf die Klasse der **ARIMA-Modelle (AutoRegressive Integrated Moving Average)**. Diese Modelle sind besonders flexibel und leistungsfähig, wenn es darum geht, komplexe Zeitreihenprozesse ohne klare saisonale Struktur zu modellieren. Die Methodik umfasst drei zentrale Schritte:

1. **Identifikation** des Modells (z. B. Wahl der ARIMA-Ordnung),
2. **Schätzung** der Modellparameter,
3. **Diagnoseprüfung** der Modellgüte.

Diese Schritte basieren auf verschiedenen statistischen Tests, Visualisierungen und Modellvergleichsmetriken wie dem **Akaike Information Criterion (AIC)** oder dem **Bayesian Information Criterion (BIC)**.

### Ziel dieser Arbeit

Wir möchten ein automatisiertes Analyseverfahren entwickeln, das:

- eine Zeitreihe auf Stationarität prüft und ggf. transformiert,
- verschiedene ARIMA-Modelle testet und bewertet,
- auf Basis statistischer Kriterien das geeignetste Modell auswählt,
- eine **Out-of-Sample-Prognose** durchführt und
- die Ergebnisse visuell und statistisch bewertet.

Zusätzlich berücksichtigen wir **Modellvalidierung** (z. B. Residuenanalyse), **t-Tests der Parameter**, sowie die **Generierung von Prognosen** inklusive **Konfidenzintervallen**.

---

Im Folgenden werden die Schritte im Detail mit zugehörigem Code und statistischem Hintergrund dokumentiert.



##  Zeitreihenanalyse – Schritt 1: Datenvorbereitung & Stationaritätsprüfung

### Überblick: Warum ist Stationarität wichtig?

Ein zentrales Ziel bei der Zeitreihenanalyse mit ARIMA-Modellen ist die Identifikation und Modellierung des zugrunde liegenden stochastischen Prozesses. ARIMA-Modelle (AutoRegressive Integrated Moving Average) setzen voraus, dass die analysierte Zeitreihe **stationär** ist.

**Stationarität** bedeutet, dass die statistischen Eigenschaften der Zeitreihe – insbesondere der Erwartungswert, die Varianz und die Autokorrelation – über die Zeit hinweg konstant bleiben. Wenn dies nicht gegeben ist (z. B. bei Trends, saisonalen Effekten oder Heteroskedastizität), kann das Modell fehlspezifiziert werden.

---

###  Datenimport & Visualisierung der Originalreihe

Wir importieren zunächst die Aktienkurs-Zeitreihe von Samsung („Adjusted Close“-Preise), konvertieren das Datum in ein geeignetes Format und erstellen eine erste Visualisierung der ursprünglichen, **nicht transformierten** Zeitreihe.


**Beobachtung:**  
Die ursprüngliche Zeitreihe zeigt visuell einen starken Trend – sowohl auf- als auch abwärtsgerichtet in unterschiedlichen Phasen. Solche Trends deuten auf **Nichtstationarität** hin.

---

###  Stationaritätstests: ADF und KPSS

Um Stationarität statistisch zu prüfen, wenden wir zwei komplementäre Tests an:

#### 🔹 Augmented Dickey-Fuller-Test (ADF)
- **Nullhypothese (H₀):** Die Zeitreihe hat eine Einheitwurzel → *nicht stationär*
- **Alternativhypothese (H₁):** Stationarität liegt vor
- Wir lehnen H₀ ab, wenn der **p-Wert < 0.05**

**Testformel:**

\[
\Delta y_t = \alpha + \beta t + \gamma y_{t-1} + \sum_{i=1}^{p} \delta_i \Delta y_{t-i} + \varepsilon_t
\]

#### 🔹 KPSS-Test (Kwiatkowski-Phillips-Schmidt-Shin)
- **Nullhypothese (H₀):** Die Zeitreihe ist stationär (gegen Trendstationarität)
- **Alternativhypothese (H₁):** Die Zeitreihe ist nicht stationär
- Wir behalten H₀ bei, wenn **p-Wert > 0.05**

**Kombination beider Tests:**  
Sie ermöglicht eine robustere Beurteilung, da sie aus zwei Perspektiven prüfen.



---

###  Transformationen zur Erreichung von Stationarität

Um die ursprüngliche Reihe in eine stationäre zu überführen, wenden wir mehrere Transformationen an – jede mit einem spezifischen Ziel:

| Transformation                        | Zweck |
|--------------------------------------|-------|
| **1. Differenzierung**: \( y_t - y_{t-1} \) | Entfernt lineare Trends |
| **2. Differenzierung**: \( (y_t - y_{t-1}) - (y_{t-1} - y_{t-2}) \) | Entfernt quadratische/komplexere Trends |
| **Logarithmierung**: \( \log(y_t) \) | Stabilisiert Varianz (z. B. bei exponentiellem Wachstum) |
| **Log-Differenz**: \( \log(y_t) - \log(y_{t-1}) \) | Kombiniert Trendentfernung und Varianzstabilisierung |
| **Moving Average Residuum**: \( y_t - \overline{y}_{t,window} \) | Entfernt gleitenden Mittelwert (Trend) |
| **Exponentielle Glättung** | Entfernt Trend mit höherem Gewicht auf jüngere Werte |
| **HP-Filter (Hodrick-Prescott)** | Trennt Trend- und Zykluskomponente der Reihe |

Nach jeder Transformation führen wir erneut ADF- und KPSS-Tests durch, um den Erfolg zu bewerten.

---

###  Visualisierung der Transformationen

Die transformierten Zeitreihen werden grafisch dargestellt – inklusive der jeweiligen Testergebnisse (ADF & KPSS) in Textboxen.

**Beispiel:**

```text
ADF p = 0.021 → Stationär  
KPSS p = 0.08 → Stationär
```

Dies erlaubt eine schnelle visuelle und numerische Bewertung jeder Transformation.

---

###  Auswahl der besten Transformation

Zur systematischen Auswahl verwenden wir eine **Scoring-Funktion**, die beide Tests kombiniert:

\[
\text{Stationaritätsscore} = p_{\text{ADF}} + (1 - p_{\text{KPSS}})
\]

- Ziel: **Minimaler Score**
- Begründung: Kleine ADF-p-Werte + große KPSS-p-Werte → stationär

Der Name und die Serie der „besten Transformation“ werden gespeichert.

---

###  Rolling-Statistiken und ACF: Visuelle Stationaritätsprüfung

Wir berechnen und visualisieren:

- **Rolling Mean** (Gleitender Durchschnitt, Fenster = 20 Tage)
- **Rolling Standard Deviation**
- **ACF-Werte bei Lag 1 & 2**

Ziel: Stationäre Reihen haben **konstante Mittelwerte und Varianzen**, und die ACF fällt schnell ab.

Für jede Transformation:

```python
rolling_mean = series.rolling(window=20).mean()
rolling_std = series.rolling(window=20).std()
acf_vals = acf(series, nlags=2)
```

---

###  ACF- und PACF-Plots

Für zwei zentrale Reihen (beste Transformation + Log-Differenz) erstellen wir **ACF- und PACF-Plots mit Konfidenzintervallen**.

#### ACF (Autokorrelationsfunktion):
Zeigt Korrelation von \( y_t \) mit \( y_{t-k} \). Wichtig für MA-Komponente im ARIMA(p, d, q).

#### PACF (Partielle Autokorrelationsfunktion):
Zeigt "direkten" Effekt des Lags \( k \) auf \( y_t \), ohne Zwischenschritte. Wichtig für AR-Komponente.

Signifikante Lags außerhalb der Konfidenzgrenzen (±1.96/√n) deuten auf relevante Modellbestandteile hin.

---

##  Fazit Schritt 1

- Die Zeitreihe wurde erfolgreich transformiert, um Stationarität zu erreichen.
- Durch Kombination von ADF- und KPSS-Tests konnte eine robuste Bewertung vorgenommen werden.
- Die **log-differenzierte Zeitreihe** erwies sich als beste Transformation.
- ACF- und PACF-Plots legen die Grundlage für die spätere Modellauswahl (ARIMA-Identifikation).

Wir sind nun bereit für **Modellidentifikation und Parameterschätzung**.


##  Schritt 2: ARIMA-Modellidentifikation via AIC/BIC

Nachdem wir im ersten Schritt die Zeitreihe erfolgreich in eine **stationäre Form** gebracht haben (z. B. durch Log-Differenzierung), können wir nun ein geeignetes ARIMA-Modell identifizieren.

---

###  Hintergrund: ARIMA(p, d, q)

Ein ARIMA-Modell kombiniert drei Komponenten:

- **AR (p)**: AutoRegressive-Teil → beschreibt, wie der aktuelle Wert von den vorherigen Werten abhängt  
- **I (d)**: Integrated-Teil → beschreibt, wie viele Differenzierungen notwendig sind, um Stationarität zu erreichen  
- **MA (q)**: Moving Average-Teil → beschreibt den Einfluss vergangener Schätzfehler (Residuen)

Die allgemeine Form eines ARIMA(p,d,q)-Modells ist:

\[
y_t = c + \phi_1 y_{t-1} + \dots + \phi_p y_{t-p} + \theta_1 \varepsilon_{t-1} + \dots + \theta_q \varepsilon_{t-q} + \varepsilon_t
\]

Dabei ist:
- \( y_t \): aktueller Wert der Zeitreihe  
- \( \phi_i \): AR-Koeffizienten  
- \( \theta_i \): MA-Koeffizienten  
- \( \varepsilon_t \): weiße Rauschkomponente (Zufallsfehler)

---

###  Ziel: Auswahl des besten (p,d,q)-Modells

Um das geeignetste Modell zu finden, wurden alle sinnvollen Kombinationen von p, d und q getestet. Die Auswahl basiert auf:

#### Bewertungskriterien:

| Kriterium | Ziel      | Formel (vereinfacht)                      | Bestrafung für Komplexität? |
|-----------|-----------|-------------------------------------------|------------------------------|
| **AIC**   | Modellgüte | \( \text{AIC} = -2 \log(L) + 2k \)         | Ja (milder)                  |
| **BIC**   | Modellgüte | \( \text{BIC} = -2 \log(L) + k \log(n) \) | Ja (stärker)                 |

- \( \log(L) \): Log-Likelihood des Modells  
- \( k \): Anzahl der geschätzten Parameter  
- \( n \): Anzahl der Beobachtungen

Ziel ist es, ein Modell mit möglichst **niedrigem AIC/BIC** zu finden. Dabei gilt:

- **AIC** bevorzugt Modelle mit besserer Vorhersagekraft (weniger Bestrafung für Komplexität)
- **BIC** bevorzugt sparsamere Modelle (mehr Bestrafung für viele Parameter)

---

###  Beispielausgabe: Top 10 Modelle

| p | d | q | AIC     | BIC     |
|---|---|---|---------|---------|
| 2 | 0 | 2 | 1234.56 | 1250.78 |
| 1 | 0 | 1 | 1240.91 | 1251.33 |
| 3 | 0 | 0 | 1243.12 | 1257.01 |
| 2 | 0 | 1 | 1244.88 | 1258.90 |
| 1 | 0 | 2 | 1246.78 | 1260.22 |

Aus dieser Tabelle erkennt man, dass das Modell mit **(p=2, d=0, q=2)** den niedrigsten AIC aufweist und damit aktuell das beste Kandidatenmodell ist.

---

###  Fazit Schritt 2

- Durch eine systematische Suche über alle Modellkombinationen wurde das ARIMA-Modell mit der besten Balance aus **Güte** und **Komplexität** identifiziert.
- Das Kriterium der Wahl war der **Akaike Information Criterion (AIC)**, optional ergänzt durch **BIC** zur Überprüfung.
- Dieses Modell kann nun im nächsten Schritt geschätzt und validiert werden.


##  Schritt 3: Visuelle Forecast-Analyse mit gestuften Trainingsdaten

Nachdem wir mit Hilfe von AIC/BIC ein vielversprechendes ARIMA(p,d,q)-Modell ausgewählt haben (z. B. ARIMA(2,0,0)), wollen wir dieses Modell nun **visuell testen**, und zwar in verschiedenen Phasen der Zeitreihe.

---

###  Ziel: Forecast-Güte über mehrere Zeitabschnitte hinweg beurteilen

Statt das Modell nur einmal auf die gesamte Serie zu trainieren, wird die Zeitreihe stufenweise aufgeteilt:

| Trainingsanteil | Forecastzeitraum | Ziel |
|------------------|------------------|------|
| 40 %             | nächste 20 %     | Frühprognose auf kurzer Datenbasis  
| 60 %             | nächste 20 %     | Stabileres Modell  
| 80 %             | nächste 20 %     | Fast vollständiges Modell  
| 100 %            | +20 % extrapoliert| Finaler Forecast über den Rand hinaus  

So können wir grafisch nachvollziehen, **wie stabil und robust** das Modell über verschiedene Zeitabschnitte funktioniert.

---

###  Methodik

1. **Split der Zeitreihe**  
   Die Serie wird schrittweise in 4 Etappen unterteilt (40 %, 60 %, 80 %, 100 %).

2. **Modellanpassung je Etappe**  
   Für jede Teilserie wird das **ARIMA(p,d,q)-Modell** neu geschätzt.

3. **Prognoseberechnung**  
   Für jede Etappe wird ein Forecast über eine feste Länge (z. B. 20 % der Gesamtlänge) berechnet.

4. **Visualisierung**  
   - Blaue Linie = Trainingsdaten  
   - Rote Linie = Prognosewerte  
   - Graue Linie = Trenngrenze zwischen Training und Forecast  

---

###  Beispielhafte Plotbeschreibung

Wenn z. B. die Zeitreihe 1000 Werte hat, ergibt sich bei 80 % Training und 20 % Forecast:

- **Trainingsbereich**: Werte 0 bis 799  
- **Forecast**: Werte 800 bis 999  
- **Ziel**: Prüfen, ob der Forecast strukturell dem tatsächlichen Verlauf folgt

> ❗ **Hinweis**: In der Praxis wurden echte zukünftige Werte nicht mitgeplottet – es geht hier primär um das **Vertrauensverhalten des Modells**, nicht um Gütemaße wie MAPE oder RMSE.

---

###  Fazit Schritt 3

- Durch die gestufte Visualisierung bekommt man ein Gefühl dafür, **wann das ARIMA-Modell stabil prognostiziert** und wo es evtl. versagt.
- Diese Methode ist **besonders hilfreich**, um **Overfitting bei kurzer Trainingsphase** zu erkennen.
- In Kombination mit der vorherigen AIC/BIC-Auswahl ist das ein robuster Weg, um die Prognosequalität eines Zeitreihenmodells visuell zu prüfen.



##  Schritt 4: Rolling Forecasts auf Originalskala (zurücktransformiert)

Nachdem wir ein geeignetes ARIMA-Modell trainiert und erste Forecasts durchgeführt haben, folgt nun der **entscheidende Praxisschritt**:  
Wir zeigen **rollierende Forecasts** – aber diesmal **auf der realen Preisskala**, nicht auf den log-differenzierten Werten.

---

###  Ziel: Modellgüte auf der ursprünglichen Skala beurteilen

Das Modell wurde auf **log-transformierten und differenzierten Daten** trainiert, da diese Transformation stationär macht.  
Jetzt kehren wir die Transformation wieder um:

1. **Kumulative Summe** der differenzierten Forecasts  
2. **Addition des letzten Log-Wertes** vor dem Forecast  
3. **Exponentialfunktion**, um von log(X) zurück zu X zu kommen

> Ergebnis: Ein Forecast der tatsächlichen Preisentwicklung – in verständlicher Skalierung (z. B. CHF oder EUR).

---

###  Methodik

#### Rolling Forecast-Stufen:

| Trainingsanteil | Forecastzeitraum | Zielsetzung |
|-----------------|------------------|-------------|
| 40 %            | nächste 20 %     | Erste Prognose auf kleiner Datenbasis  
| 60 %            | nächste 20 %     | Etwas robuster  
| 80 %            | nächste 20 %     | Fast vollständige Datenlage  
| 100 %           | +20 % extrapoliert| Zukunftsprognose über das bekannte Ende hinaus  

#### Vorgehen pro Stufe:

- **Modelltraining** auf einem festen Anteil der geloggten Serie
- **Forecast** über die nächsten 20 % (ab Trainingsende)
- **Zurücktransformation**:
  - Letzter Log-Wert + kumulierte Differenzen
  - Dann: `exp()` anwenden → Originalwerte
- **Plot-Vergleich**:
  - Hellgrau = vollständige Originalserie
  - Rot = Forecast (auf Originalskala)
  - Blau (gestrichelt) = Tatsächliche Entwicklung (sofern bekannt)

---

###  Visualisierungsziel

Ein Plot pro Stufe zeigt:

- Wie **gut das Modell zukünftige Entwicklungen approximiert**
- Ob es **Trendwenden erkennt**
- Ob es **überschätzt oder unterschätzt**
- Wo es z. B. **zu langsam reagiert**

Die letzte Grafik (100 %) zeigt die echte **Zukunftsprognose** – also was wir erwarten würden, wenn der bisherige Verlauf anhielte.

---

###  Fazit Schritt 4

- Forecasts auf Originalskala ermöglichen realistische Bewertungen – **"Was kostet es in CHF?"** statt "Wie ist der log-Wert?"
- Die Kombination aus Training in Stufen + zurücktransformierten Forecasts ist ideal zur **Modellvalidierung & Kommunikation**
- Die rollierenden Forecastplots sind auch für **Stakeholder außerhalb der Datenanalyse** sehr anschaulich

---

 **Tipp**:  
Im nächsten Schritt lohnt sich ein Blick auf **Residualanalyse und Diagnostik**, um zu überprüfen, ob Modellannahmen (z. B. weiße Rauschen im Fehler) erfüllt sind.


##  Schritt 5: Residuenanalyse & Modell-Diagnose (ARIMA)

Ein statistisches Modell wie ARIMA ist nur dann sinnvoll, wenn seine **Fehler (Residuen)** bestimmte Kriterien erfüllen.

---

###  Ziel der Residuenanalyse:

Ein **gutes Modell** hinterlässt **"weißes Rauschen"** – also zufällige, unkorrelierte Fehler ohne Muster.  
Deshalb prüfen wir in diesem Schritt:

1. **Sind die Fehler zufällig verteilt?**
2. **Weisen sie Normalverteilung auf?**
3. **Gibt es Autokorrelation in den Fehlern?**
4. **Sind die Modellparameter signifikant (t-Tests)?**

---

###  Die 4 diagnostischen Plots

| Plot | Beschreibung | Ziel |
|------|--------------|------|
| **Residuen-Zeitreihe** | Fehler über die Zeit | Keine offensichtlichen Muster, keine Trendlinie |
| **Histogramm + Normalverteilung** | Vergleich Fehlerverteilung vs. Normalverteilung | Möglichst symmetrisch und glockenförmig |
| **Q-Q-Plot (Quantil-Quantil)** | Prüft Normalverteilung grafisch | Punkte sollten auf Diagonale liegen |
| **ACF (Autokorrelationsfunktion)** | Gibt es systematische Fehler über die Zeit? | Alle Werte im Konfidenzintervall (kein Muster) |

---

###  Statistische Tests

#### 1.  Modell-Zusammenfassung (`model_fit.summary()`)

- Zeigt **Koeffizienten** und deren **t-Statistiken**
- Wenn `|t| > 2` → oft statistisch signifikant
- AIC/BIC sind ebenfalls enthalten

#### 2.  Ljung-Box-Test

- Testet, ob Residuen **autokorreliert** sind
- Nullhypothese: Keine Autokorrelation vorhanden
- Wenn `p > 0.05` → kein signifikanter Zusammenhang → **gut!**

> Beispielausgabe:
> ```
>                 lb_stat  lb_pvalue
> lag
> 10          8.764328    0.552399
> ```
> Interpretation: Bei Lag 10 kein Hinweis auf systematische Fehler (p > 0.05)

---

###  Fazit Schritt 5

- **Bestätigt die statistische Güte** des Modells
- Zeigt, ob die Modellannahmen verletzt sind
- Grundlage für Vertrauen in Prognoseergebnisse

Wenn **Residuen Muster zeigen**, solltest du dein Modell (p,d,q) noch einmal überdenken.

---

 **Nächster Schritt (optional):**  
Vergleich mit anderen Modellen (z. B. SARIMA, Prophet) oder Forecasting auf echter Preisskala in der Zukunft.
